# CLX DGA Detection

This is an introduction to CLX DGA Detection.

## What is DGA Detection?

[Domain Generation Algorithms](https://en.wikipedia.org/wiki/Domain_generation_algorithm) (DGAs) are used to generate domain names that can be used by the malware to communicate with the command and control servers. IP addresses and static domain names can be easily blocked, and a DGA provides an easy method to generate a large number of domain names and rotate through them to circumvent traditional block lists.

## When to use CLX DGA Detection?

Use CLX DGA Detection to build your own DGA Detection model that can then be used to predict whether a given domain is malicious or not. We will use a type of recurrent neural network called the Gated Recurrent Unit (GRU) for this example. The [CLX](https://github.com/rapidsai/clx) and [RAPIDS](https://rapids.ai/) libraries enable users train their models with up-to-date domain names representative of both benign and DGA generated strings. Using a [CLX workflow](./intro-clx-workflow.html), this capability could also be used in production environments.  
  
**For a more advanced, in-depth example of CLX DGA Detection view this Jupyter** [notebook](https://github.com/rapidsai/clx/blob/master/notebooks/dga_detection/DGA_Detection.ipynb).

## How to train a CLX DGA Detection model

To train a CLX DGA Detection model you simply need a training data set which contains a column of domains and their associated `type` which can be either `1` (benign) or `0` (malicious).

First initialize your new model

In [2]:
LR = 0.001
N_LAYERS = 3
CHAR_VOCAB = 128
HIDDEN_SIZE = 100
N_DOMAIN_TYPE = 2 # Will be 2 since there are a total of 2 different types

from clx.analytics.dga_detector import DGADetector
dd = DGADetector(lr=LR)
dd.init_model(n_layers=N_LAYERS, char_vocab=CHAR_VOCAB, hidden_size=HIDDEN_SIZE, n_domain_type=N_DOMAIN_TYPE)

Next, train your DGA detector. The below example uses a small dataset for demonstration only. Ideally you will want a larger training set.

To develop a more expansive training set, these resources are available:
  
* DGA : http://osint.bambenekconsulting.com/feeds/dga-feed.txt
* Benign : http://s3.amazonaws.com/alexa-static/top-1m.csv.zip

In [3]:
import cudf

train_df1 = cudf.DataFrame()
train_df1['domain'] = ['google.com', 'youtube.com', 'tmall.com', 'duiwlqeejymdb.com', 'kofsmyaiufarb.net', 'xskphhmrlcihr.biz']
train_df1['type'] = [1,1,1,0,0,0]

train_df2 = cudf.DataFrame()
train_df2['domain'] = ['yahoo.com', 'linkedin.com', 'twitter.com', 'wejaecjhycwss.co.uk', 'xtorhktvpblmr.info', 'xvljisbfalkts.com']
train_df2['type'] = [1,1,1,0,0,0]

# Convert domains from string to ascii
train_df1_ascii = dd.str2ascii(train_df2, 6)
train_df2_ascii = dd.str2ascii(train_df2, 6)
train_df2_ascii

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,len,type,domain
3,119,101,106,97,101,99,106,104,121,99,...,115,46,99,111,46,117,107,19,0,wejaecjhycwss.co.uk
4,120,116,111,114,104,107,116,118,112,98,...,114,46,105,110,102,111,0,18,0,xtorhktvpblmr.info
5,120,118,108,106,105,115,98,102,97,108,...,115,46,99,111,109,0,0,17,0,xvljisbfalkts.com
1,108,105,110,107,101,100,105,110,46,99,...,0,0,0,0,0,0,0,12,1,linkedin.com
2,116,119,105,116,116,101,114,46,99,111,...,0,0,0,0,0,0,0,11,1,twitter.com
0,121,97,104,111,111,46,99,111,109,0,...,0,0,0,0,0,0,0,9,1,yahoo.com


When we train a model, the total loss is returned

In [6]:
total_num_of_domains = train_df1['domain'].count() + train_df2['domain'].count()
dd.train_model([train_df1_ascii, train_df2_ascii], total_num_of_domains)

0.005933642387390137

Ideally, you will want to train your model over a number of `epochs` as detailed in our example DGA Detection [notebook](https://github.com/rapidsai/clx/blob/master/notebooks/dga_detection/DGA_Detection.ipynb).

## Save a trained model

In [7]:
dd.save_model("clx_dga_classifier.pth")

## Load a model

Let's create a new dga detector and load the saved model from above.

In [8]:
dga_detector = DGADetector(lr=0.001)
dga_detector.load_model("clx_dga_classifier.pth")

## DGA Inferencing

Use your new model to predict malicious domains

In [10]:
test_df = cudf.DataFrame()
test_df['domain'] = ['facebook.com','ylqblbltqkynb.net']

dga_detector.predict(test_df['domain'])

0    1
1    0
Name: is_dga, dtype: int64

## Conclusion

DGA detector in CLX enables users to train their models for detection and also use existing models. This capability could also be used in conjunction with log parsing efforts if the logs contain domain names. DGA detection done with CLX and RAPIDS keeps data in GPU memory, removing unnecessary copy/converts and providing a 4X speed advantage over CPU only implementations. This is esepcially true with large batch sizes.